In [5]:
from google.colab import files

uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving ratings.txt to ratings.txt
User uploaded file "ratings.txt" with length 19515078 bytes


In [7]:
import pandas as pd
import io

nsmc_fname = 'ratings.txt'
nsmc_total_data = pd.read_csv(io.StringIO(uploaded[nsmc_fname].decode('utf-8')), sep='\t')
text_data_list = nsmc_total_data['document'].dropna().values

print(text_data_list[:4])

['어릴때보고 지금다시봐도 재밌어요ㅋㅋ'
 '디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산업이 부러웠는데. 사실 우리나라에서도 그 어려운시절에 끝까지 열정을 지킨 노라노 같은 전통이있어 저와 같은 사람들이 꿈을 꾸고 이뤄나갈 수 있다는 것에 감사합니다.'
 '폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.'
 '와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런게 진짜 영화지']


In [11]:
from glove import Corpus, Glove
from konlpy.tag import Okt; okt=Okt()

text_data = [['<OOV>'] + okt.morphs(xx) for xx in text_data_list]

corpus = Corpus() 
corpus.fit(text_data, window=5)

glove_model = Glove(no_components=256, learning_rate=0.05)
glove_model.fit(corpus.matrix, epochs=10, no_threads=4, verbose=True)
glove_model.add_dictionary(corpus.dictionary)

Performing 10 training epochs with 4 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9


In [24]:
print(len(glove_model.dictionary))

print(glove_model.word_vectors[glove_model.dictionary['평점']].shape)
print(glove_model.most_similar('평점', number=5))

122829
(256,)
[('조작', 0.9279802660479385), ('네이버', 0.8963304541646842), ('낮추기', 0.8619796688785405), ('조정', 0.8389010081978051)]


In [25]:
import numpy as np

print(text_data_list[0])
print(okt.morphs(text_data_list[0]))

test_glove_embedd = []
for xx in okt.morphs(text_data_list[0]):
    try:
        test_glove_embedd.append(glove_model.word_vectors[glove_model.dictionary[xx]])
    except KeyError:
        test_glove_embedd.append(glove_model.word_vectors[glove_model.dictionary['<OOV>']])
        
print(np.asarray(test_glove_embedd).shape)

어릴때보고 지금다시봐도 재밌어요ㅋㅋ
['어릴', '때', '보고', '지금', '다시', '봐도', '재밌어요', 'ㅋㅋ']
(8, 256)
